In [1]:
from utils import load_data
dataset = load_data('../data/dev-v1.1-processed.json')

Load 10570 examples from ../data/dev-v1.1-processed.json...


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = dataset[55: 85]

In [4]:
def get_word(passage, start, end):
    from preprocess import SpacyTokenizer
    mine = SpacyTokenizer()
    tokenized = [i[0] for i in mine.tokenize(passage).data]
    joined = ' '. join(tokenized[start : end + 1])
    return joined

In [5]:
print('Question:', data[0].question)
print('Answer:', get_word(data[0].passage, data[0].y_start.index(1), data[0].y_end.index(1)))

Question: Who was the Super Bowl 50 MVP ?
Answer: Von Miller


In [6]:
class custom_arguments:
    def __init__(self, args):
        self.batch_size = args['batch_size']
        self.doc_layers = args['doc_layers']
        self.dropout_emb = args['dropout_emb']
        self.dropout_rnn_output = args['dropout_rnn_output']
        self.embedding_file = args['embedding_file']
        self.epoch = args['epoch']
        self.finetune_topk = args['finetune_topk']
        self.gpu = args['gpu']
        self.grad_clipping = args['grad_clipping']
        self.hidden_size = args['hidden_size']
        self.lr = args['lr']
        self.ner_emb_dim = args['ner_emb_dim']
        self.optimizer = args['optimizer']
        self.pos_emb_dim = args['pos_emb_dim']
        self.pretrained = args['pretrained']
        self.rel_emb_dim = args['rel_emb_dim']
        self.rnn_padding = args['rnn_padding']
        self.rnn_type = args['rnn_type']
        self.seed = args['seed']
        self.test_mode = args['test_mode']
        self.use_cuda = args['use_cuda']
        self.p_max_size = args['p_max_size']
        
        
args_dict = {'batch_size':32, 'doc_layers':1, 'dropout_emb':0.4, 'dropout_rnn_output':0.4,
             'embedding_file':'/home/unchartech001/Local_Resources/glove.6B/glove.6B.300d.txt',
             'epoch':50, 'finetune_topk':10, 'gpu':'0', 'grad_clipping':10.0, 'hidden_size':96,
             'lr':0.002, 'ner_emb_dim':8, 'optimizer':'adamax', 'pos_emb_dim':12, 'pretrained':'',
             'rel_emb_dim':10, 'rnn_padding':True, 'rnn_type':'lstm', 'seed':1234, 'test_mode':False,
             'use_cuda':True, 'p_max_size':750}
args = custom_arguments(args_dict)

In [7]:
%%capture
import model
from utils import build_vocab
build_vocab()
model = model.Model(args)

In [8]:
prediction = model.predict(data)
len(prediction)

xWy torch.Size([32, 750, 750])
x_mask torch.Size([32, 750])
xWy torch.Size([32, 750, 750])
x_mask torch.Size([32, 750])
attn_map_weights_start torch.Size([32, 750, 750])
attn_map_start torch.Size([32, 750, 1])


RuntimeError: size mismatch, m1: [24000 x 1], m2: [750 x 750] at /pytorch/aten/src/TH/generic/THTensorMath.c:2033

In [ ]:
prediction[0].size()

In [ ]:
import torch
def output_to_map(prediction):
    start_maps, end_maps = [], []
    for sub_start, sub_end in zip(prediction[0], prediction[1]):
        start = int(torch.argmax(sub_start))
        end = int(torch.argmax(sub_end))
        start_maps.append([0] * start + [1] + [0]*(len(prediction[0]) - start))
        end_maps.append([0] * end + [1] + [0]*(len(prediction[0]) - end))
    return start_maps, end_maps

In [ ]:
def map_padding(start, end):
    padded = []
    padded.extend([0] * start.index(1))
    padded.extend([1] * (end.index(1) - start.index(1) + 1))
    padded.extend([0] * (len(start) - end.index(1) - 1))
    return padded

In [ ]:
def evaluate(model, dev_data):
    from sklearn.metrics import f1_score
    f1_scores = []
    for batch_input in model._iter_data(dev_data):
            print(batch_input)
            feed_input = [x for x in batch_input[:-1]]
            pred_proba = model.network(*feed_input)
            print(pred_proba)
            map_pred = output_to_map(pred_proba)
            for i, data in enumerate(feed_input):
                print(data)
                truth = map_padding(data.y_start, data.y_end)
                prediction = map_padding(map_pred[0][i], map_pred[1][i])
                f1_scores += f1_score(truth, prediction)
    return sum(f1_scores)/len(f1_scores)

In [ ]:
import torch
m = torch.nn.Bilinear(1, 1, 7)

input1 = torch.randn(4, 7, 1)
input2 = torch.randn(4, 7, 1)
output = m(input1, input2)
print(output.size())